In [34]:
%load_ext autoreload
%autoreload 2

from tweedejaars_project import load_df
from tweedejaars_project import show_real_penalty_score
from tweedejaars_project.evaluation.metrics import compute_time_diff_flip
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.linear_model import *


df = load_df()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
features = [
    "import_capacity",
    "settlement_price_bestguess",
]

train_x = df[features][:165195]
train_y = df['target_two_sided_ptu'][:165195]

test_x = df[features][165195:]
test_y = df['target_two_sided_ptu'][165195:]

# Initialize the Decision Tree Classifier
clf = HistGradientBoostingClassifier()

# Train the classifier
clf.fit(train_x, train_y)

y_pred = clf.predict(test_x)

# Evaluate the model
accuracy = accuracy_score(test_y, y_pred)
print(accuracy)

print("Classification Report:")
print(classification_report(test_y, y_pred))

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(test_y, y_pred))

0.9758950791084631
Classification Report:
              precision    recall  f1-score   support

       False       0.98      0.99      0.99     54028
        True       0.12      0.05      0.07      1023

    accuracy                           0.98     55051
   macro avg       0.55      0.52      0.53     55051
weighted avg       0.97      0.98      0.97     55051

Confusion Matrix:
[[53674   354]
 [  973    50]]


In [36]:
show_real_penalty_score(df, df['target_two_sided_ptu'], df['target_two_sided_ptu'], df['ptu_id'])

0/4901, 0/20384


(0, 4901, 0, 20384)

In [39]:
compute_time_diff_flip(df, df['target_two_sided_ptu'], df['ptu_id'])

              neg         pos
count  529.000000  183.000000
mean   -10.293006    1.377049
std      2.616087    1.605252
min    -14.000000    0.000000
25%    -13.000000    0.000000
50%    -10.000000    1.000000
75%     -8.000000    2.000000
max     -4.000000    9.000000
-7.293539325842697/-105853.30357142857, 712/1456


(count    529.000000
 mean     -10.293006
 std        2.616087
 min      -14.000000
 25%      -13.000000
 50%      -10.000000
 75%       -8.000000
 max       -4.000000
 dtype: float64,
 count    183.000000
 mean       1.377049
 std        1.605252
 min        0.000000
 25%        0.000000
 50%        1.000000
 75%        2.000000
 max        9.000000
 dtype: float64,
 -7.293539325842697,
 712,
 -105853.30357142857,
 1456)

In [38]:
false_neg_test_df = {
    'Time': [
        '11:30', '11:31', '11:32', '11:33', '11:34', '11:35', '11:36', 
        '11:37', '11:38', '11:39', '11:40', '11:41', '11:42', '11:43', 
        '11:44', '11:45'
    ],
    'min_price_published': [
        -100, np.NaN, 0, -100, -150, 
        -200, -300, -100, np.NaN, np.NaN, 
        np.NaN, np.NaN, np.NaN, -30, -100, 0
    ],
    'max_price_published': [
        np.NaN, 20, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
        40, 50, 90, 200, 80, 
        np.NaN, np.NaN, np.NaN
    ],
    'settlement_price_bestguess': [
        np.NaN, np.NaN, 0, -100, -150, -200, -300, -300, 
        40, 50, 90, 200, 200, 
        200, 200, 200
    ],
    'target_two_sided_ptu': [
        False, False, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True
    ],
    'pred': [
        True, False, False, True, True, False, False, True, False, False,
        False, False, False, False, False, False
    ],
    'naive_strategy_action': [
        False, False, False, True, True, True, True, True, False, False,
        False, False, False, False, False, False,
    ],
    'ptu_id':
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
}

false_pos_test_df = {
    'Time': [
        '11:30', '11:31', '11:32', '11:33', '11:34', '11:35', '11:36', 
        '11:37', '11:38', '11:39', '11:40', '11:41', '11:42', '11:43', 
        '11:44', '11:45'
    ],
    'min_price_published': [
        -100, -100, 0, -100, -150, 
        -200, -300, -100, -100, -100, 
        -100, -100, -100, -30, -100, 0
    ],
    'max_price_published': [
        np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
        np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
        np.NaN, np.NaN, np.NaN
    ],
    'settlement_price_bestguess': [
        np.NaN, np.NaN, 0, -100, -150, -200, -300, -300, 
        -300, -300, -300, -300, -300, 
        -300, -300, -300
    ],
    'target_two_sided_ptu': [
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False
    ],
    'pred': [
        True, False, False, True, True, False, False, True, False, False,
        False, False, False, False, False, False
    ],
    'naive_strategy_action': [
        False, False, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True,
    ],
    'ptu_id':
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
}

false_neg_test_df = pd.DataFrame(false_neg_test_df)
false_pos_test_df = pd.DataFrame(false_pos_test_df)

false_neg = show_real_penalty_score(false_neg_test_df, false_neg_test_df['target_two_sided_ptu'], false_neg_test_df['pred'], false_neg_test_df['ptu_id'])
false_pos = show_real_penalty_score(false_pos_test_df, false_pos_test_df['target_two_sided_ptu'], false_pos_test_df['pred'], false_pos_test_df['ptu_id'])
print(false_neg)
print(false_pos)

2/5, 0/0
0/0, 3/14
(2, 5, 0, 0)
(0, 0, 3, 14)
